In [9]:
from pyensembl import EnsemblRelease

genome = EnsemblRelease(77)
genome.download()
genome.index()

INFO:pyensembl.download_cache:Fetching /home/ec2-user/.cache/pyensembl/GRCh38/ensembl77/Homo_sapiens.GRCh38.77.gtf.gz from URL https://ftp.ensembl.org/pub/release-77/gtf/homo_sapiens/Homo_sapiens.GRCh38.77.gtf.gz
INFO:datacache.download:Downloading https://ftp.ensembl.org/pub/release-77/gtf/homo_sapiens/Homo_sapiens.GRCh38.77.gtf.gz to /home/ec2-user/.cache/pyensembl/GRCh38/ensembl77/Homo_sapiens.GRCh38.77.gtf.gz
INFO:pyensembl.download_cache:Fetching /home/ec2-user/.cache/pyensembl/GRCh38/ensembl77/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL https://ftp.ensembl.org/pub/release-77/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
INFO:datacache.download:Downloading https://ftp.ensembl.org/pub/release-77/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz to /home/ec2-user/.cache/pyensembl/GRCh38/ensembl77/Homo_sapiens.GRCh38.cdna.all.fa.gz
INFO:pyensembl.download_cache:Fetching /home/ec2-user/.cache/pyensembl/GRCh38/ensembl77/Homo_sapiens.GRCh38.ncrna.fa.gz from URL htt

In [11]:
label = 'ENSG00000002079'

In [18]:
genome.gene_ids()[0]

'ENSG00000000003'

In [19]:
gene = genome.gene_by_id('ENSG00000000003')

# Observed and predicted gene expression values were obtained by summing up the observed/predicted CAGE read counts
# at all unique TSS locations of the gene. For each TSS location, we used the 128-bp bin overlapping the TSS as well
# as the two neighboring bins (3 bins in total).

# For each gene, look through all transcipts - protein coding / not, and record offsets from gene start
gene.transcripts

[Transcript(transcript_id='ENST00000612152', transcript_name='TSPAN6-201', gene_id='ENSG00000000003', biotype='protein_coding', contig='X', start=100627109, end=100637104, strand='-', genome='GRCh38'),
 Transcript(transcript_id='ENST00000373020', transcript_name='TSPAN6-001', gene_id='ENSG00000000003', biotype='protein_coding', contig='X', start=100628670, end=100636806, strand='-', genome='GRCh38'),
 Transcript(transcript_id='ENST00000614008', transcript_name='TSPAN6-202', gene_id='ENSG00000000003', biotype='protein_coding', contig='X', start=100632063, end=100637104, strand='-', genome='GRCh38'),
 Transcript(transcript_id='ENST00000496771', transcript_name='TSPAN6-003', gene_id='ENSG00000000003', biotype='processed_transcript', contig='X', start=100632541, end=100636689, strand='-', genome='GRCh38'),
 Transcript(transcript_id='ENST00000494424', transcript_name='TSPAN6-002', gene_id='ENSG00000000003', biotype='processed_transcript', contig='X', start=100633442, end=100639991, strand='

In [13]:
len(genome.protein_ids())

99436

In [14]:
protein_id = genome.protein_ids()[0]
genome.protein_sequence(protein_id)

# results = []
# for protein_id in genome.protein_ids():
#     sequence = genome.protein_sequence(protein_id)
#     if '*' not in sequence:
#         results.append({
#             'protein_id': protein_id,
#             'protein_sequence': sequence
#         })

'MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTIPTIGFNVETVEYKNICFTVWDVGGQDKIRPLWRHYFQNTQGLIFVVDSNDRERVQESADELQKMLQEDELRDAVLLVFANKQDMPNAMPVSELTDKLGLQHLRSRTWYVQATCATQGTGLYDGLDWLSHELSKR'

In [20]:
gene.transcripts[0]

Transcript(transcript_id='ENST00000612152', transcript_name='TSPAN6-201', gene_id='ENSG00000000003', biotype='protein_coding', contig='X', start=100627109, end=100637104, strand='-', genome='GRCh38')

In [21]:
gene

Gene(gene_id='ENSG00000000003', gene_name='TSPAN6', biotype='protein_coding', contig='X', start=100627109, end=100639991, strand='-', genome='GRCh38')

In [22]:
!pip install pybiomart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 4.6 MB/s eta 0:00:00


In [24]:
from pybiomart import Dataset as PyBiomartDataset

dataset = PyBiomartDataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')

In [26]:
# dataset.list_attributes().to_clipboard()

In [29]:
genes_proteins = dataset.query(attributes=['ensembl_gene_id', 'ensembl_peptide_id',
                                            'gene_biotype', 'transcript_is_canonical', 'peptide'], filters={})

In [34]:
genes_proteins.to_csv('gene2protein.csv')

In [33]:
genes_proteins['Peptide']

0         MRPQELPRLAFPLLLLLLLLLPPPPCPAHSATRFDPTWESLDARQL...
1         MQRSPLEKASVVSKLFFSWTRPILRKGYRQRLELSDIYQIPSVDSA...
2         MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...
3         MKAARFVLRSAGSLNGAGLVPREVEHFSRYSPSPLSMKQLLDFGSE...
4         MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...
                                ...                        
189778    MAVARVDAALPPGEGSVVNWSGQGLQKLGPNLPCEADIHTLILDKN...
189779    MNGEADCPTDLEMAAPKGQDRWSQEDMLTLLECMKNNLPSNDSSKF...
189780    MASEDIAKLAETLAKTQVAGGQLSFKGKSLKLNTAEDAKDVIKEIE...
189781    MGLLSDPVRRRALARLVLRLNAPLCVLSYVAGIAWFLALVFPPLTQ...
189782    XESCQNFYKDFTLQIDMAFNVFFLLYFGLRFIAANDKLWFWLEVNS...
Name: Peptide, Length: 189783, dtype: object

In [35]:
1

1